# Lab 1: Basic Prompt Engineering

**Week 2 - Prompt Engineering & LLM Basics**

**Provided by:** ADC ENGINEERING & CONSULTING LTD

## Objectives

In this lab, you will:
- Master the fundamentals of prompt engineering
- Understand prompt components (instructions, context, examples, constraints)
- Practice different prompt styles and formats
- Learn to optimize prompts iteratively
- Handle common prompting challenges
- Build a reusable prompt library
- Evaluate and measure prompt effectiveness

## Prerequisites

- Completed Week 1 labs
- OpenAI API key configured
- Understanding of API parameters
- Python 3.9+

## Setup and Installation

In [ ]:
# Install required packages
!pip install openai python-dotenv tiktoken pandas --quiet

In [ ]:
import os
import json
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
from datetime import datetime

from openai import OpenAI
from dotenv import load_dotenv
import tiktoken

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("✓ Setup complete!")

## Part 1: Anatomy of a Good Prompt

A well-structured prompt typically contains these components:

1. **Instruction**: What you want the model to do
2. **Context**: Background information or setting
3. **Input Data**: The specific data to process
4. **Output Format**: How you want the response structured
5. **Constraints**: Limitations or requirements
6. **Examples**: Sample inputs/outputs (optional but powerful)

Let's explore each component:

### Component 1: Instructions

Clear, specific instructions are the foundation of good prompts.

In [ ]:
def test_instructions():
    """Compare vague vs specific instructions."""
    
    # Example 1: Vague instruction
    vague_prompt = "Tell me about Python."
    
    # Example 2: Specific instruction
    specific_prompt = "Explain the three main differences between lists and tuples in Python, with code examples for each difference."
    
    prompts = [
        ("Vague", vague_prompt),
        ("Specific", specific_prompt)
    ]
    
    for label, prompt in prompts:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300
        )
        
        print(f"\n{'='*80}")
        print(f"{label} Instruction:")
        print(f"{'='*80}")
        print(f"Prompt: {prompt}")
        print(f"\nResponse:\n{response.choices[0].message.content}")
        print(f"\nTokens used: {response.usage.total_tokens}")

test_instructions()

### Exercise 1.1: Improve Vague Instructions

Transform these vague instructions into specific, actionable prompts:

In [ ]:
# TODO: Improve these vague instructions

vague_instructions = [
    "Write about AI.",
    "Make it better.",
    "Explain this code.",
    "Help me with my email.",
    "What should I do?"
]

# Example transformation:
# Vague: "Write about AI"
# Specific: "Write a 3-paragraph explanation of how neural networks work, suitable for a high school student with no programming background."

specific_instructions = [
    # TODO: Add your improved versions here
    "",  # Improved version of "Write about AI"
    "",  # Improved version of "Make it better"
    "",  # Improved version of "Explain this code"
    "",  # Improved version of "Help me with my email"
    "",  # Improved version of "What should I do?"
]

# Test your improved instructions
# for i, instruction in enumerate(specific_instructions):
#     if instruction:
#         response = client.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[{"role": "user", "content": instruction}],
#             temperature=0.7,
#             max_tokens=200
#         )
#         print(f"\n{i+1}. {instruction}")
#         print(f"Response: {response.choices[0].message.content}\n")

### Component 2: Context

Providing relevant context helps the model understand the situation and generate appropriate responses.

In [ ]:
def compare_with_without_context():
    """Compare responses with and without context."""
    
    # Without context
    no_context = "Should I use async or sync functions?"
    
    # With context
    with_context = """
    Context: I'm building a web API with FastAPI that needs to handle 1000+ 
    concurrent requests. The API makes database queries and calls external APIs.
    
    Question: Should I use async or sync functions for my route handlers?
    """
    
    prompts = [
        ("Without Context", no_context),
        ("With Context", with_context)
    ]
    
    for label, prompt in prompts:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=250
        )
        
        print(f"\n{'='*80}")
        print(f"{label}:")
        print(f"{'='*80}")
        print(response.choices[0].message.content)

compare_with_without_context()

### Exercise 1.2: Add Context

Add relevant context to make these questions more answerable:

In [ ]:
# TODO: Add context to these questions

questions_needing_context = [
    "What's the best database to use?",
    "How should I structure my code?",
    "Is this a good approach?",
    "What framework should I choose?",
]

# Example:
# Original: "What's the best database to use?"
# With Context: """
# Context: I'm building a real-time chat application that needs to:
# - Handle 10,000 concurrent users
# - Store message history
# - Support full-text search
# - Deploy on AWS
# 
# Question: What's the best database to use?
# """

contextualized_questions = [
    # TODO: Add your contextualized versions
]

# Test your questions
# for question in contextualized_questions:
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[{"role": "user", "content": question}],
#         temperature=0.7
#     )
#     print(f"\nQuestion:\n{question}")
#     print(f"\nResponse:\n{response.choices[0].message.content}\n")
#     print("="*80)

### Component 3: Output Format

Specifying the desired output format ensures consistent, parseable responses.

In [ ]:
def test_output_formats():
    """Test different output format specifications."""
    
    base_instruction = "List the top 5 programming languages for web development."
    
    formats = {
        "Unstructured": base_instruction,
        
        "Numbered List": base_instruction + "\n\nFormat: Provide as a numbered list.",
        
        "Table": base_instruction + "\n\nFormat: Provide as a markdown table with columns: Language, Primary Use, Difficulty Level.",
        
        "JSON": base_instruction + """
        
        Format: Return as JSON array with objects containing:
        - name: language name
        - use_case: primary use case
        - difficulty: beginner/intermediate/advanced
        """,
        
        "Structured": base_instruction + """
        
        Format:
        For each language provide:
        1. Language name (bold)
        2. One-sentence description
        3. Best use case
        4. Popularity score (1-10)
        """
    }
    
    for format_name, prompt in formats.items():
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=300
        )
        
        print(f"\n{'='*80}")
        print(f"Format: {format_name}")
        print(f"{'='*80}")
        print(response.choices[0].message.content)

test_output_formats()

### Exercise 1.3: Design Output Formats

Create prompts with specific output formats for these tasks:

In [ ]:
# TODO: Create prompts with specific output formats

tasks = {
    "task1": {
        "description": "Get a weather summary for a week",
        "prompt": "",  # TODO: Add prompt with JSON format
        "expected_format": "JSON"
    },
    "task2": {
        "description": "Compare three products",
        "prompt": "",  # TODO: Add prompt with table format
        "expected_format": "Markdown Table"
    },
    "task3": {
        "description": "Create a study plan",
        "prompt": "",  # TODO: Add prompt with structured format
        "expected_format": "Structured with sections"
    },
    "task4": {
        "description": "Extract key information from text",
        "prompt": "",  # TODO: Add prompt with bullet points
        "expected_format": "Bullet points"
    }
}

# Test your prompts
# for task_name, task_info in tasks.items():
#     if task_info["prompt"]:
#         response = client.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[{"role": "user", "content": task_info["prompt"]}],
#             temperature=0.3
#         )
#         print(f"\n{task_name} - {task_info['description']}")
#         print(f"Expected format: {task_info['expected_format']}")
#         print(f"\nResponse:\n{response.choices[0].message.content}\n")
#         print("="*80)

### Component 4: Constraints

Constraints guide the model to stay within desired boundaries.

In [ ]:
def test_constraints():
    """Test the impact of constraints."""
    
    base_task = "Explain quantum computing"
    
    constraints_examples = [
        {
            "name": "No constraints",
            "prompt": base_task
        },
        {
            "name": "Length constraint",
            "prompt": f"{base_task} in exactly 3 sentences."
        },
        {
            "name": "Audience constraint",
            "prompt": f"{base_task} for a 10-year-old child."
        },
        {
            "name": "Style constraint",
            "prompt": f"{base_task} using only simple words (no jargon)."
        },
        {
            "name": "Multiple constraints",
            "prompt": f"{base_task} in exactly 50 words, using an analogy, without technical terms."
        }
    ]
    
    for example in constraints_examples:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": example["prompt"]}],
            temperature=0.7,
            max_tokens=200
        )
        
        print(f"\n{'='*80}")
        print(f"Constraint: {example['name']}")
        print(f"{'='*80}")
        print(f"Prompt: {example['prompt']}")
        print(f"\nResponse:\n{response.choices[0].message.content}")
        print(f"\nWord count: {len(response.choices[0].message.content.split())}")

test_constraints()

### Exercise 1.4: Apply Constraints

Add appropriate constraints to these prompts:

In [ ]:
# TODO: Add constraints to these prompts

prompts_to_constrain = [
    {
        "task": "Explain machine learning",
        "original": "Explain machine learning.",
        "constraints_to_add": ["length: 4 sentences", "no equations", "use everyday analogy"],
        "improved": ""  # TODO: Add your improved version
    },
    {
        "task": "Write a product description",
        "original": "Write a product description for wireless headphones.",
        "constraints_to_add": ["max 100 words", "highlight 3 key features", "call-to-action"],
        "improved": ""  # TODO: Add your improved version
    },
    {
        "task": "Summarize an article",
        "original": "Summarize this article about climate change.",
        "constraints_to_add": ["5 bullet points", "focus on solutions", "cite statistics"],
        "improved": ""  # TODO: Add your improved version
    }
]

# Test your constrained prompts
# for item in prompts_to_constrain:
#     if item["improved"]:
#         print(f"\nTask: {item['task']}")
#         print(f"Original: {item['original']}")
#         print(f"Constraints: {', '.join(item['constraints_to_add'])}")
#         print(f"Improved: {item['improved']}\n")
#         
#         response = client.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[{"role": "user", "content": item["improved"]}],
#             temperature=0.7
#         )
#         print(f"Response:\n{response.choices[0].message.content}\n")
#         print("="*80)

## Part 2: Prompt Patterns

Let's explore common, reusable prompt patterns.

### Pattern 1: Persona Pattern

Assign a specific role or persona to the model.

In [ ]:
def test_persona_pattern():
    """Test different persona patterns."""
    
    question = "How should I prepare for a job interview?"
    
    personas = [
        {
            "name": "Career Coach",
            "system_message": "You are an experienced career coach with 15 years of experience helping people land their dream jobs."
        },
        {
            "name": "HR Manager",
            "system_message": "You are an HR manager at a Fortune 500 company who has conducted over 1000 interviews."
        },
        {
            "name": "Recruitment Consultant",
            "system_message": "You are a recruitment consultant specializing in tech roles, known for giving practical, no-nonsense advice."
        }
    ]
    
    for persona in personas:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": persona["system_message"]},
                {"role": "user", "content": question}
            ],
            temperature=0.7,
            max_tokens=250
        )
        
        print(f"\n{'='*80}")
        print(f"Persona: {persona['name']}")
        print(f"{'='*80}")
        print(response.choices[0].message.content)

test_persona_pattern()

### Exercise 2.1: Create Persona Prompts

Design persona-based prompts for these scenarios:

In [ ]:
# TODO: Design persona prompts

scenarios = [
    {
        "scenario": "Code review feedback",
        "user_message": "Review this Python function for potential improvements.",
        "personas": [
            {"name": "Senior Developer", "system_message": ""},  # TODO
            {"name": "Security Expert", "system_message": ""},  # TODO
            {"name": "Performance Engineer", "system_message": ""}  # TODO
        ]
    },
    {
        "scenario": "Learning advice",
        "user_message": "How should I learn data science?",
        "personas": [
            {"name": "University Professor", "system_message": ""},  # TODO
            {"name": "Self-taught Data Scientist", "system_message": ""},  # TODO
            {"name": "Bootcamp Instructor", "system_message": ""}  # TODO
        ]
    }
]

# Test your personas
# for scenario_info in scenarios:
#     print(f"\n{'='*80}")
#     print(f"Scenario: {scenario_info['scenario']}")
#     print(f"{'='*80}")
#     
#     for persona in scenario_info['personas']:
#         if persona['system_message']:
#             response = client.chat.completions.create(
#                 model="gpt-3.5-turbo",
#                 messages=[
#                     {"role": "system", "content": persona['system_message']},
#                     {"role": "user", "content": scenario_info['user_message']}
#                 ],
#                 temperature=0.7
#             )
#             print(f"\nPersona: {persona['name']}")
#             print(f"Response: {response.choices[0].message.content}\n")

### Pattern 2: Template Pattern

Use templates with placeholders for consistency.

In [ ]:
class PromptTemplate:
    """
    A reusable prompt template with placeholders.
    """
    
    def __init__(self, template: str, description: str = ""):
        """
        Initialize template.
        
        Args:
            template: Template string with {placeholders}
            description: Description of the template's purpose
        """
        self.template = template
        self.description = description
    
    def format(self, **kwargs) -> str:
        """
        Fill in the template with values.
        
        Args:
            **kwargs: Values for placeholders
        
        Returns:
            Formatted prompt
        """
        return self.template.format(**kwargs)
    
    def get_placeholders(self) -> List[str]:
        """Extract placeholder names from template."""
        import re
        return re.findall(r'\{(\w+)\}', self.template)

# Create some useful templates
templates = {
    "summarizer": PromptTemplate(
        template="""
        Summarize the following {content_type} in {style} style.
        Length: {length}
        Focus on: {focus_areas}
        
        Content:
        {content}
        """,
        description="Generic summarization template"
    ),
    
    "code_explainer": PromptTemplate(
        template="""
        Explain this {language} code to a {audience_level} audience.
        Focus on: {focus}
        Include: {include_items}
        
        Code:
        ```{language}
        {code}
        ```
        """,
        description="Code explanation template"
    ),
    
    "text_transformer": PromptTemplate(
        template="""
        Transform the following text:
        - From style: {from_style}
        - To style: {to_style}
        - Maintain: {maintain}
        - Change: {change}
        
        Text:
        {text}
        """,
        description="Text style transformation template"
    )
}

# Test the summarizer template
summarizer = templates["summarizer"]
print(f"Template: {summarizer.description}")
print(f"Placeholders: {summarizer.get_placeholders()}")

prompt = summarizer.format(
    content_type="article",
    style="executive",
    length="3 bullet points",
    focus_areas="key findings and recommendations",
    content="Machine learning models require large amounts of data for training..."
)

print(f"\nFormatted prompt:\n{prompt}")

# Use the prompt
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

print(f"\nResponse:\n{response.choices[0].message.content}")

### Exercise 2.2: Build Your Template Library

Create reusable templates for common tasks:

In [ ]:
# TODO: Create your own prompt templates

my_templates = {
    "email_writer": PromptTemplate(
        template="""
        # TODO: Create a template for writing emails
        # Placeholders might include: {purpose}, {tone}, {recipient}, {key_points}
        """,
        description="Email writing template"
    ),
    
    "bug_analyzer": PromptTemplate(
        template="""
        # TODO: Create a template for analyzing bugs
        # Placeholders might include: {error_message}, {code_snippet}, {context}
        """,
        description="Bug analysis template"
    ),
    
    "content_repurposer": PromptTemplate(
        template="""
        # TODO: Create a template for repurposing content
        # Placeholders might include: {original_format}, {target_format}, {content}
        """,
        description="Content repurposing template"
    ),
    
    "learning_path": PromptTemplate(
        template="""
        # TODO: Create a template for generating learning paths
        # Placeholders might include: {topic}, {current_level}, {goal}, {timeframe}
        """,
        description="Learning path generator template"
    )
}

# Test your templates
# for name, template in my_templates.items():
#     print(f"\nTemplate: {name}")
#     print(f"Description: {template.description}")
#     print(f"Placeholders: {template.get_placeholders()}")
#     
#     # Fill in with your test values and get response
#     # prompt = template.format(...)
#     # response = client.chat.completions.create(...)

### Pattern 3: Instruction-Example Pattern

Combine instructions with examples for better results.

In [ ]:
def test_instruction_example_pattern():
    """Test instruction with examples vs without."""
    
    task = "Extract the sentiment (positive/negative/neutral) and key topics from customer reviews."
    
    # Without examples
    prompt_no_examples = f"""
    Task: {task}
    
    Review: "The product arrived late and the packaging was damaged, but the item itself works perfectly."
    """
    
    # With examples
    prompt_with_examples = f"""
    Task: {task}
    
    Examples:
    
    Review: "Love this product! Fast shipping and great quality."
    Sentiment: Positive
    Topics: product quality, shipping speed
    
    Review: "Terrible experience. Product broke after one day."
    Sentiment: Negative
    Topics: product durability, reliability
    
    Review: "It's okay, does what it says but nothing special."
    Sentiment: Neutral
    Topics: product functionality
    
    Now analyze this review:
    Review: "The product arrived late and the packaging was damaged, but the item itself works perfectly."
    """
    
    prompts = [
        ("Without Examples", prompt_no_examples),
        ("With Examples", prompt_with_examples)
    ]
    
    for label, prompt in prompts:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        
        print(f"\n{'='*80}")
        print(f"{label}:")
        print(f"{'='*80}")
        print(response.choices[0].message.content)

test_instruction_example_pattern()

## Part 3: Iterative Prompt Improvement

Learn to systematically improve prompts.

In [ ]:
@dataclass
class PromptVersion:
    """Track different versions of a prompt."""
    version: int
    prompt: str
    response: str
    tokens: int
    evaluation: str
    improvements: List[str]

class PromptOptimizer:
    """
    Systematically optimize prompts.
    """
    
    def __init__(self):
        self.versions: List[PromptVersion] = []
    
    def test_prompt(self, prompt: str, evaluation_criteria: str = "") -> PromptVersion:
        """
        Test a prompt and record results.
        
        Args:
            prompt: The prompt to test
            evaluation_criteria: How to evaluate the response
        
        Returns:
            PromptVersion object with results
        """
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        
        version = PromptVersion(
            version=len(self.versions) + 1,
            prompt=prompt,
            response=response.choices[0].message.content,
            tokens=response.usage.total_tokens,
            evaluation=evaluation_criteria,
            improvements=[]
        )
        
        self.versions.append(version)
        return version
    
    def compare_versions(self):
        """Compare all tested versions."""
        print(f"\n{'='*80}")
        print("PROMPT VERSION COMPARISON")
        print(f"{'='*80}\n")
        
        for v in self.versions:
            print(f"Version {v.version}")
            print(f"Tokens: {v.tokens}")
            print(f"Prompt: {v.prompt[:100]}...")
            print(f"Response: {v.response[:150]}...")
            print(f"Evaluation: {v.evaluation}")
            if v.improvements:
                print(f"Improvements made: {', '.join(v.improvements)}")
            print(f"\n{'-'*80}\n")
    
    def get_best_version(self, criteria: str = "shortest") -> PromptVersion:
        """
        Get the best version based on criteria.
        
        Args:
            criteria: 'shortest' (fewest tokens) or 'latest'
        """
        if criteria == "shortest":
            return min(self.versions, key=lambda v: v.tokens)
        return self.versions[-1]

# Example: Optimize a prompt iteratively
optimizer = PromptOptimizer()

# Version 1: Basic prompt
v1 = optimizer.test_prompt(
    "Explain recursion",
    evaluation_criteria="Too vague, lacks specificity"
)
v1.improvements = []

# Version 2: Add specificity
v2 = optimizer.test_prompt(
    "Explain recursion in programming with an example",
    evaluation_criteria="Better, but no format specification"
)
v2.improvements = ["Added example request"]

# Version 3: Add format
v3 = optimizer.test_prompt(
    """Explain recursion in programming.
    
    Format:
    1. Definition (1 sentence)
    2. Simple code example
    3. Explanation of how it works
    4. When to use it (1 sentence)
    """,
    evaluation_criteria="Good structure, clear output format"
)
v3.improvements = ["Added format specification", "Structured output"]

# Version 4: Add constraints
v4 = optimizer.test_prompt(
    """Explain recursion in programming for a beginner.
    
    Format:
    1. Definition (1 sentence, no jargon)
    2. Simple Python code example (factorial function)
    3. Step-by-step explanation of execution
    4. When to use it vs loops (1 sentence)
    
    Keep the total explanation under 150 words.
    """,
    evaluation_criteria="Excellent: audience-specific, constrained, structured"
)
v4.improvements = ["Added audience level", "Specific example", "Length constraint"]

# Compare all versions
optimizer.compare_versions()

# Get best version
best = optimizer.get_best_version("latest")
print(f"\n{'='*80}")
print(f"BEST VERSION (Version {best.version}):")
print(f"{'='*80}")
print(f"Prompt:\n{best.prompt}\n")
print(f"Response:\n{best.response}")

### Exercise 3.1: Optimize a Prompt

Take this basic prompt through 4 iterations of improvement:

In [ ]:
# TODO: Optimize this prompt through iterations

# Starting prompt
basic_prompt = "Write a function to sort a list"

# Create your optimizer
my_optimizer = PromptOptimizer()

# Version 1: Original (test it)
# v1 = my_optimizer.test_prompt(basic_prompt, "Baseline - too vague")

# Version 2: TODO - Add specificity
# What language? What sorting algorithm? What should the function signature be?
# v2_prompt = ""
# v2 = my_optimizer.test_prompt(v2_prompt, "")
# v2.improvements = [""]

# Version 3: TODO - Add constraints and format
# Add error handling? Input validation? Documentation?
# v3_prompt = ""
# v3 = my_optimizer.test_prompt(v3_prompt, "")
# v3.improvements = [""]

# Version 4: TODO - Add examples and edge cases
# Provide example inputs/outputs? Handle edge cases?
# v4_prompt = ""
# v4 = my_optimizer.test_prompt(v4_prompt, "")
# v4.improvements = [""]

# Compare and analyze
# my_optimizer.compare_versions()

## Part 4: Common Prompt Challenges

Learn to handle typical problems.

### Challenge 1: Handling Ambiguity

In [ ]:
def handle_ambiguity():
    """Demonstrate handling ambiguous prompts."""
    
    # Ambiguous prompt
    ambiguous = "How do I fix the error?"
    
    # Clarified prompt
    clarified = """
    I'm getting this Python error:
    
    ```
    TypeError: unsupported operand type(s) for +: 'int' and 'str'
    ```
    
    In this code:
    ```python
    age = 25
    message = "Age: " + age
    ```
    
    How do I fix this error?
    """
    
    prompts = [
        ("Ambiguous", ambiguous),
        ("Clarified", clarified)
    ]
    
    for label, prompt in prompts:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        
        print(f"\n{'='*80}")
        print(f"{label}:")
        print(f"{'='*80}")
        print(response.choices[0].message.content)

handle_ambiguity()

### Challenge 2: Preventing Hallucinations

Guide the model to admit uncertainty rather than fabricate information.

In [ ]:
def prevent_hallucinations():
    """Techniques to reduce hallucinations."""
    
    question = "What are the key features of the Python 4.0 release?"
    # Note: Python 4.0 doesn't exist - testing if model will hallucinate
    
    # Without guidance
    no_guidance = question
    
    # With uncertainty instruction
    with_guidance = f"""
    {question}
    
    Important: If you're not certain about the information or if Python 4.0 hasn't 
    been released yet, please say so clearly rather than speculating.
    """
    
    # With date constraint
    with_date = f"""
    Based on information available up to your knowledge cutoff date:
    {question}
    
    If Python 4.0 hasn't been released yet, please state that clearly.
    """
    
    prompts = [
        ("No Guidance", no_guidance),
        ("With Guidance", with_guidance),
        ("With Date Constraint", with_date)
    ]
    
    for label, prompt in prompts:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        
        print(f"\n{'='*80}")
        print(f"{label}:")
        print(f"{'='*80}")
        print(response.choices[0].message.content)

prevent_hallucinations()

### Challenge 3: Maintaining Consistency

Ensure consistent formatting and style across responses.

In [ ]:
def ensure_consistency():
    """Use system messages and templates for consistency."""
    
    # Inconsistent responses
    questions = [
        "What is Python?",
        "What is JavaScript?",
        "What is SQL?"
    ]
    
    print("WITHOUT CONSISTENCY MEASURES:")
    print("="*80)
    for q in questions:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": q}],
            temperature=0.7
        )
        print(f"\n{q}")
        print(response.choices[0].message.content[:150] + "...")
    
    # With consistency measures
    print("\n\nWITH CONSISTENCY MEASURES:")
    print("="*80)
    
    system_message = """
    You are a programming language expert. When asked about a programming language, 
    always respond in this exact format:
    
    **Language:** [Name]
    **Type:** [Type of language]
    **Primary Use:** [Main use case]
    **Key Feature:** [One distinguishing feature]
    **Difficulty:** [Beginner/Intermediate/Advanced]
    """
    
    for q in questions:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": q}
            ],
            temperature=0.7
        )
        print(f"\n{q}")
        print(response.choices[0].message.content)

ensure_consistency()

### Exercise 4.1: Fix Problematic Prompts

Identify and fix issues in these prompts:

In [ ]:
# TODO: Fix these problematic prompts

problematic_prompts = [
    {
        "original": "Tell me everything about databases",
        "problem": "Too broad, overwhelming",
        "fixed": ""  # TODO: Make it more focused
    },
    {
        "original": "Is machine learning better than deep learning?",
        "problem": "False comparison, they're related not alternatives",
        "fixed": ""  # TODO: Reframe the question properly
    },
    {
        "original": "How much does a car cost?",
        "problem": "Needs context and constraints",
        "fixed": ""  # TODO: Add relevant context
    },
    {
        "original": "Make my code faster",
        "problem": "No code provided, no context",
        "fixed": ""  # TODO: Add code and context
    }
]

# Test your fixed prompts
# for item in problematic_prompts:
#     if item["fixed"]:
#         print(f"\nOriginal Problem: {item['problem']}")
#         print(f"Original: {item['original']}")
#         print(f"Fixed: {item['fixed']}\n")
#         
#         response = client.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[{"role": "user", "content": item["fixed"]}],
#             temperature=0.5
#         )
#         print(f"Response:\n{response.choices[0].message.content}\n")
#         print("="*80)

## Part 5: Building a Prompt Library

Create a reusable collection of tested prompts.

In [ ]:
class PromptLibrary:
    """
    Manage a library of tested, reusable prompts.
    """
    
    def __init__(self):
        self.prompts: Dict[str, Dict] = {}
    
    def add_prompt(
        self,
        name: str,
        template: str,
        category: str,
        description: str,
        example_inputs: Dict = None,
        tags: List[str] = None
    ):
        """Add a prompt to the library."""
        self.prompts[name] = {
            "template": template,
            "category": category,
            "description": description,
            "example_inputs": example_inputs or {},
            "tags": tags or [],
            "created_at": datetime.now().isoformat(),
            "usage_count": 0
        }
    
    def get_prompt(self, name: str) -> Optional[Dict]:
        """Retrieve a prompt from the library."""
        prompt = self.prompts.get(name)
        if prompt:
            prompt["usage_count"] += 1
        return prompt
    
    def search_prompts(self, query: str = "", category: str = "", tag: str = "") -> List[Dict]:
        """Search prompts by query, category, or tag."""
        results = []
        
        for name, prompt in self.prompts.items():
            match = True
            
            if query and query.lower() not in name.lower() and query.lower() not in prompt["description"].lower():
                match = False
            
            if category and prompt["category"] != category:
                match = False
            
            if tag and tag not in prompt["tags"]:
                match = False
            
            if match:
                results.append({"name": name, **prompt})
        
        return results
    
    def list_categories(self) -> List[str]:
        """List all categories."""
        return list(set(p["category"] for p in self.prompts.values()))
    
    def export_to_json(self, filepath: str):
        """Export library to JSON file."""
        with open(filepath, 'w') as f:
            json.dump(self.prompts, f, indent=2)
        print(f"✓ Library exported to {filepath}")
    
    def import_from_json(self, filepath: str):
        """Import library from JSON file."""
        with open(filepath, 'r') as f:
            self.prompts = json.load(f)
        print(f"✓ Library imported from {filepath}")

# Create and populate a library
library = PromptLibrary()

# Add some prompts
library.add_prompt(
    name="code_reviewer",
    template="""
    Review the following {language} code for:
    - Code quality and best practices
    - Potential bugs or issues
    - Performance considerations
    - Security concerns
    
    Code:
    ```{language}
    {code}
    ```
    
    Provide specific, actionable feedback.
    """,
    category="code",
    description="Comprehensive code review prompt",
    example_inputs={"language": "python", "code": "def example(): pass"},
    tags=["code", "review", "quality"]
)

library.add_prompt(
    name="learning_path_generator",
    template="""
    Create a personalized learning path for: {topic}
    
    Current level: {current_level}
    Goal: {goal}
    Time available: {timeframe}
    Learning style: {learning_style}
    
    Provide:
    1. Learning phases (beginner → advanced)
    2. Key concepts to master in order
    3. Recommended resources for each phase
    4. Practical projects to build
    5. Estimated timeline
    """,
    category="education",
    description="Generate structured learning paths",
    example_inputs={
        "topic": "Machine Learning",
        "current_level": "beginner",
        "goal": "build ML models",
        "timeframe": "3 months",
        "learning_style": "hands-on"
    },
    tags=["education", "learning", "career"]
)

library.add_prompt(
    name="bug_analyzer",
    template="""
    Analyze this bug:
    
    Error: {error_message}
    
    Code context:
    ```{language}
    {code}
    ```
    
    Environment: {environment}
    
    Provide:
    1. Root cause analysis
    2. Step-by-step solution
    3. Prevention tips
    4. Related issues to watch for
    """,
    category="debugging",
    description="Systematic bug analysis",
    example_inputs={
        "error_message": "TypeError: ...",
        "code": "...",
        "language": "python",
        "environment": "Python 3.9, Ubuntu 20.04"
    },
    tags=["debugging", "troubleshooting", "code"]
)

# Search the library
print("\nAll prompts in 'code' category:")
code_prompts = library.search_prompts(category="code")
for p in code_prompts:
    print(f"- {p['name']}: {p['description']}")

print("\nAll prompts with 'learning' tag:")
learning_prompts = library.search_prompts(tag="learning")
for p in learning_prompts:
    print(f"- {p['name']}: {p['description']}")

# Use a prompt from the library
prompt_info = library.get_prompt("code_reviewer")
if prompt_info:
    template = prompt_info["template"]
    filled_prompt = template.format(
        language="python",
        code="""
def calculate_total(items):
    total = 0
    for item in items:
        total = total + item
    return total
        """
    )
    
    print(f"\n{'='*80}")
    print("Using prompt from library: code_reviewer")
    print(f"{'='*80}")
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": filled_prompt}],
        temperature=0.3
    )
    
    print(response.choices[0].message.content)

# Export library
library.export_to_json("my_prompt_library.json")

### Exercise 5.1: Build Your Own Prompt Library

Create a library with at least 5 prompts for different use cases:

In [ ]:
# TODO: Build your personal prompt library

my_library = PromptLibrary()

# TODO: Add prompts for:
# 1. Email writing (professional, casual, follow-up)
# 2. Content creation (blog posts, social media, documentation)
# 3. Data analysis (summarize findings, generate insights)
# 4. Problem solving (brainstorming, decision making)
# 5. Communication (meeting notes, status updates, explanations)

# Example:
# my_library.add_prompt(
#     name="professional_email",
#     template="...",
#     category="communication",
#     description="...",
#     tags=["email", "professional"]
# )

# Test your library
# my_library.list_categories()
# my_library.search_prompts(category="communication")
# my_library.export_to_json("personal_prompt_library.json")

## Challenge Projects

### Challenge 1: Prompt A/B Testing Framework

Build a system to test multiple prompt variations and compare results:

In [ ]:
class PromptABTester:
    """
    A/B test different prompt variations.
    
    TODO: Implement:
    1. Create prompt variations
    2. Run tests with multiple inputs
    3. Collect metrics (tokens, quality, consistency)
    4. Statistical comparison
    5. Generate report with winner
    """
    
    def __init__(self):
        self.tests = []
    
    # TODO: Implement methods
    
    pass

# Usage example:
# tester = PromptABTester()
# tester.add_variation("v1", "Explain {topic}")
# tester.add_variation("v2", "Explain {topic} in simple terms")
# tester.run_test(inputs=[{"topic": "AI"}, {"topic": "ML"}])
# tester.get_report()

### Challenge 2: Prompt Quality Analyzer

Create a tool that analyzes prompts and suggests improvements:

In [ ]:
class PromptQualityAnalyzer:
    """
    Analyze prompt quality and suggest improvements.
    
    TODO: Implement:
    1. Check for common issues (vagueness, ambiguity, missing context)
    2. Analyze structure and completeness
    3. Suggest specific improvements
    4. Provide quality score
    5. Generate improved version
    """
    
    def __init__(self):
        self.checks = []
    
    # TODO: Implement analysis methods
    
    pass

# Usage example:
# analyzer = PromptQualityAnalyzer()
# report = analyzer.analyze("Tell me about Python")
# print(report["issues"])
# print(report["suggestions"])
# print(report["improved_prompt"])

### Challenge 3: Domain-Specific Prompt Generator

Build a generator that creates optimized prompts for specific domains:

In [ ]:
class DomainPromptGenerator:
    """
    Generate domain-specific prompts.
    
    TODO: Implement for domains like:
    1. Healthcare (medical Q&A, diagnosis support)
    2. Legal (contract analysis, legal research)
    3. Education (lesson plans, quiz generation)
    4. Business (market analysis, strategy)
    5. Technical (code generation, system design)
    
    Each domain should have:
    - Specialized templates
    - Domain-appropriate constraints
    - Expert personas
    - Compliance considerations
    """
    
    def __init__(self, domain: str):
        self.domain = domain
        self.templates = {}
    
    # TODO: Implement domain-specific generation
    
    pass

# Usage example:
# generator = DomainPromptGenerator(domain="healthcare")
# prompt = generator.generate("patient_symptoms_analysis",
#                            symptoms=["fever", "cough"],
#                            duration="3 days")

## Summary

In this lab, you've learned:

1. ✅ The anatomy of well-structured prompts
2. ✅ Essential prompt components (instructions, context, format, constraints)
3. ✅ Common prompt patterns (persona, template, instruction-example)
4. ✅ Iterative prompt improvement methodology
5. ✅ Handling common challenges (ambiguity, hallucinations, consistency)
6. ✅ Building and managing a prompt library

### Key Takeaways

- **Specificity matters**: Vague prompts yield vague results
- **Context is crucial**: Background information improves accuracy
- **Format your outputs**: Structured responses are more useful
- **Use constraints**: Guide the model with clear boundaries
- **Iterate systematically**: Test, measure, improve
- **Build reusable assets**: Create a library of tested prompts
- **Consistency requires structure**: Use templates and system messages

### Best Practices

1. **Start simple, iterate**: Begin with basic prompt, refine based on results
2. **Be specific**: Clear instructions > vague requests
3. **Provide examples**: Show don't just tell
4. **Structure outputs**: Specify exactly how you want responses formatted
5. **Add constraints**: Length, style, content boundaries
6. **Use templates**: Consistency and reusability
7. **Test variations**: A/B test different approaches
8. **Document what works**: Build your prompt library

### Next Steps

- Complete the challenge projects
- Build your personal prompt library with 20+ prompts
- Move on to Lab 2: Few-Shot Learning Experiments
- Practice prompt engineering daily

**Provided by:** ADC ENGINEERING & CONSULTING LTD